In [1]:
import carla
import random
import time
import numpy as np
import cv2

In [2]:
def process_img(image):
    i = np.array(image.raw_data)  # convert to an array
    i2 = i.reshape((IM_HEIGHT, IM_WIDTH, 4))  # was flattened, so we're going to shape it.
    i3 = i2[:, :, :3]  # remove the alpha (basically, remove the 4th index  of every pixel. Converting RGBA to RGB)
    cv2.imshow("", i3)  # show it.
    cv2.waitKey(1)
    return i3/255.0  # normalize

In [3]:
IM_WIDTH = 640
IM_HEIGHT = 480

In [ ]:
actor_list = []
try:
    client = carla.Client("localhost", 2000)
    #client.set_timeout(2.0)
    world = client.get_world()
    blueprint_library = world.get_blueprint_library() #incorporate new actors into the simulation
    bp = blueprint_library.filter("model3")[0]
    print(bp)
    spawn_point = world.get_map().get_spawn_points()
    vehicle = world.spawn_actor(bp, random.choice(spawn_point))
    #vehicle.set_autopilot(True)
    vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0.0))
    actor_list.append(vehicle)

    cam_bp = blueprint_library.find("sensor.camera.rgb")
    # change the dimensions of the image
    cam_bp.set_attribute('image_size_x', f'{IM_WIDTH}')
    cam_bp.set_attribute('image_size_y', f'{IM_HEIGHT}')
    cam_bp.set_attribute('fov', '110')
    
    # Adjust sensor relative to vehicle
    spawn_point = carla.Transform(carla.Location(x=2.5, z=0.7))

    # spawn the sensor and attach to vehicle.
    sensor = world.spawn_actor(cam_bp, spawn_point, attach_to=vehicle)

    # add sensor to list of actors
    actor_list.append(sensor)
    
    sensor.listen(lambda data: process_img(data))
    
    
    time.sleep(100)

    
finally:
    for actor in actor_list:
        actor.destroy()
    print("all cleaned up!!")

ActorBlueprint(id=vehicle.tesla.model3,tags=[tesla, model3, vehicle])
